In [1]:
# bob.refresh {"cron": "15 9-23 * * 1-5", "timeout": 400000}

In [2]:
import bob
hc = bob.HealthCheck()
hc_key = "bfc41c24-d278-4bc8-8d95-767e5da44f59"
start = hc.start(hc_key)
print(start)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/bfc41c24-d278-4bc8-8d95-767e5da44f59, 2020-04-17


In [3]:
%run __init__.ipynb

Successfully connected to MongoDB


In [4]:
#-- Data source
# ACTUAL
db_yahoo = bob.mongo.read_df('DB_ALL','yahoo').drop('_id', axis=1).sort_values(by='DATE_ORDER',ascending=False)
db_all = pd.DataFrame()

# Get only the last 360 days
tickers = db_yahoo['TICKER'].drop_duplicates().values.tolist()
for ticker in tickers:
    tmp_df = db_yahoo[db_yahoo['TICKER'] == ticker]
    all_dates = tmp_df['DATE'].drop_duplicates().sort_values(ascending=False).values.tolist()[:360]
    tmp_df = tmp_df[tmp_df['DATE'].isin(all_dates)]
    db_all = db_all.append(tmp_df)

# ML    
db_all_ml = bob.mongo.read_df('DB_ALL_ML','yahoo').drop('_id', axis=1).sort_values(by='DATE_ORDER',ascending=False)
# db_all_ml.to_csv(output_folder +'DB_ALL_ML.csv', sep=';')

In [5]:
#-- Transform yahoo finance data DB_ALL (VALUE, VAR, VARP)
def get_all(df,df_ml,filter1=None):
    #-- ACUTAL DATA
    #Filter data
    if filter1 == None:
        domain = df.reset_index(drop=True)
    else:
        domain = df[df['TYPE'] == filter1]
        
    domain_ac = pd.DataFrame({
        'ENTITY': 'WORLDWIDE',
        'SCENARIO': pd.to_datetime(domain['DATE'], format='%Y-%m-%d').dt.strftime('%d/%m/%Y'),
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['LABEL'],
        'GROUPS': 'ACTUAL',
        'SETTING': "w/o ML",
        'TYPE': domain['TYPE'],
        'VALUE': domain['CLOSE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'VOLUME': domain['VOLUME'].astype('int'),
        'META': domain['META'],
        'UNIT_VALUE': domain['UNIT'],
        'UNIT_VARP': ' %',
    })
    
    #-- ML DATA
    #Filter data
    if filter1 == None:
        domain = df_ml.reset_index(drop=True)
    else:
        domain = df_ml[df_ml['TYPE'] == filter1].reset_index(drop=True)  
        
    domain_ml = pd.DataFrame({
        'ENTITY': 'WORLDWIDE',
        'SCENARIO': pd.to_datetime(domain['DATE'], format='%Y-%m-%d').dt.strftime('%d/%m/%Y'),
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['LABEL'],
        'GROUPS': domain['ML'],
        'SETTING': "with ML",
        'TYPE': domain['TYPE'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'META': domain['META'],
        'UNIT_VALUE': domain['UNIT'],
        'UNIT_VARP': ' %',
    })
    
    #Prepare to save
    domain = pd.concat([domain_ac,domain_ml], axis=0).reset_index(drop=True)
    domain['LAST_UPDATE']=datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain = domain.sort_values(by=['DATE_ORDER','ENTITY','TYPE'], ascending=False).reset_index(drop=True)
    return domain

In [6]:
def save_df_all(domain,domain_num):
    #Save to mongo
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("SCENARIO", DESCENDING)])
    df_collection.create_index([("LABEL", ASCENDING)])
    return domain

In [7]:
#-- Economy Highlighted KPIS 
domain200 = get_all(db_all,db_all_ml)
domain200 = domain200[domain200['GROUPS'] == 'ACTUAL']
domain200 = save_df_all(domain200,'200')
# domain200

Dataframe 200 successfully save in database app-wsr in MongoDB. Time: --- 18.756577968597412 secnds ---


In [8]:
#-- Indice Performance
domain201bis = get_all(db_all,db_all_ml,'INDICES')
domain201bis = save_df_all(domain201bis,'201BIS')
# domain201bis

Dataframe 201BIS successfully save in database app-wsr-dev in MongoDB. Time: --- 0.3380744457244873 secnds ---


In [9]:
#-- Major Currencies
domain202bis = get_all(db_all,db_all_ml,'CURRENCIES')
domain202bis = save_df_all(domain202bis,'202BIS')
# domain202bis

Dataframe 202BIS successfully save in database app-wsr-dev in MongoDB. Time: --- 7.9925148487091064 secnds ---


In [10]:
#Stories - Market winners / market losers
def domain_203bis(df,domain_num):
    #-- Get market winners and market losers
    df=df[~df['UNIT_VALUE'].isin([' KRW',' CNY'])]
    limit = 7
    domain = pd.DataFrame()
    all_dates = df[['DATE_ORDER','GROUPS']].sort_values(by=['DATE_ORDER'], ascending=False).drop_duplicates().values.tolist()
    for d in all_dates: 
        date=d[0]
        groups=d[1]
        #Filter on date
        tmp_df = df[(df['DATE_ORDER'] == date) & (df['GROUPS'] == groups)]
        
        #Get winner, sort var. in % descending
        df_winner= tmp_df[tmp_df['VARP'] > 0].sort_values(by='VARP',ascending=False)[:limit]
        df_winner['PACKS'] = 'MARKET WINNERS'
        
        #Get losers, sort var. in % ascending
        df_loser= tmp_df[tmp_df['VARP'] < 0].sort_values(by='VARP',ascending=True)[:limit]
        df_loser['PACKS'] = 'MARKET LOSERS'
        
        #Append domain
        domain = domain.append(df_winner).append(df_loser)
        
    #-- Create DF for Toucan
    domain = pd.DataFrame({
        'ENTITY': 'WORLDWIDE',
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['LABEL'],
        'GROUPS': domain['GROUPS'],
        'SETTING': domain['SETTING'],
        'PACKS': domain['PACKS'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VAR'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'LAST_UPDATE': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
    })      
    
    #Save dataframe
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("DATE_ORDER", DESCENDING), ("PACKS", DESCENDING)])
    
    return domain

domain203allbis = get_all(db_all,db_all_ml,'EQUITIES')
domain203bis = domain_203bis(domain203allbis,'203BIS')
# domain203bis

Dataframe 203BIS successfully save in database app-wsr-dev in MongoDB. Time: --- 0.3423576354980469 secnds ---


In [11]:
#-- Commodities
#Data source
def domain_204bis(df,domain_num):    
    #Create df for all categories
    df['PACKS'] = df['META']
    
    df_all = df.copy()
    df_all['META'] = ' All categories'
    domain = pd.concat([df,df_all])

    #Save to mongo
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("SCENARIO", DESCENDING)])
    df_collection.create_index([("LABEL", ASCENDING)])
    df_collection.create_index([("META", ASCENDING)])
    return domain

domain204bis = get_all(db_all,db_all_ml,'COMMODITIES')
domain204bis = domain_204bis(domain204bis,'204BIS')
# domain204bis

Dataframe 204BIS successfully save in database app-wsr-dev in MongoDB. Time: --- 7.942807197570801 secnds ---


In [12]:
def max_date(df,domain_num):
    #get max date for each ticker
    df = df[df['GROUPS'] == 'ACTUAL']
    domain = pd.DataFrame()
    
    tickers = df['LABEL'].drop_duplicates().sort_values(ascending=True).values.tolist()
#     print(tickers)
    for ticker in tickers:
#         print(ticker)
        tmp_df = df[df['LABEL'] == ticker]
        max_df = tmp_df['DATE_ORDER'].max()
#         print(max_df)
        tmp_df = tmp_df[tmp_df['DATE_ORDER'] == max_df]
        domain = domain.append(tmp_df)
        
    #Save to mongo
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("SCENARIO", DESCENDING)])
    df_collection.create_index([("LABEL", ASCENDING)])
    return domain

In [13]:
#-- Indice Performance (Widget)
domain021 = max_date(domain201bis,'021')
# domain021

Dataframe 021 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.02323293685913086 secnds ---


In [14]:
#-- Major Currencies
domain022 = max_date(domain202bis,'022')
# domain022

Dataframe 022 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.022048234939575195 secnds ---


In [15]:
#-- Equities
domain023 = max_date(domain203allbis,'023')
# domain023

Dataframe 023 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.03186368942260742 secnds ---


In [16]:
#-- Commodities
domain024 = max_date(domain204bis,'024')
# domain024

Dataframe 024 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.02780914306640625 secnds ---


In [17]:
#-- Data source
# ACTUAL
db_trend = bob.mongo.read_df('DB_TREND','yahoo')

# ML
db_trend_ml = bob.mongo.read_df('DB_TREND_ML','yahoo')
db_trend_ml.to_csv(output_folder +'DB_TREND_ML.csv', sep=';')

In [18]:
#-- Transform yahoo finance data DB_TREND
def get_trend(df,df_ml,variable):
    domain = df.copy()[df['TYPE'] == variable]
    domain_ac = pd.DataFrame({
        'ENTITY': 'WORLDWIDE',
        'DATE': domain['DATE'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['LABEL'],
        'GROUPS': 'ACTUAL',
        'BOTTOM_FILTER': domain['DATE_SCENARIO'],
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': "w/o ML",
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
        'OPEN': domain['OPEN'],
        'HIGH': domain['HIGH'],
        'LOW': domain['LOW'],
        'CLOSE': domain['CLOSE'],
        'VOLUME': domain['VOLUME'],
        'META': domain['META'],
        'META1': domain['META1'],
        'META2': domain['META2'],
    })
    
    domain = df_ml.copy()[(df_ml['TYPE'] == variable) & (df_ml['ML'] != 'ACTUAL')]
    domain_ml = pd.DataFrame({
        'ENTITY': 'WORLDWIDE',
        'DATE': domain['DATE'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['LABEL'],
        'GROUPS':  domain['ML'],
        'BOTTOM_FILTER': domain['DATE_SCENARIO'],
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': "with ML",
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
        'META': domain['META'],
        'META1': domain['META1'],
        'META2': domain['META2'],
    })
    
    domain_ml2 = domain_ac.copy()
    domain_ml2['UPPER_FILTER_R'] = "with ML"
    
    domain = pd.concat([domain_ac,domain_ml,domain_ml2], axis=0).reset_index(drop=True)
    domain['LAST_UPDATE']=datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    return domain

In [19]:
def save_df(domain,domain_num):
    #Save dataframe
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("LABEL", ASCENDING),("BOTTOM_FILTER", DESCENDING),("UPPER_FILTER_M", DESCENDING),("UPPER_FILTER_R", DESCENDING)])
    return domain

In [20]:
#-- Economy - Indice
domain201 = save_df(get_trend(db_trend, db_trend_ml,'INDICES'),'201')
# domain201

Dataframe 201 successfully save in database app-wsr-dev in MongoDB. Time: --- 23.351306915283203 secnds ---


In [21]:
#-- Economy - Currencies
domain202 = save_df(get_trend(db_trend, db_trend_ml, 'CURRENCIES'),'202')
# domain202

upper_filter_202 = domain202[['META','LABEL']].drop_duplicates().sort_values(by=['META','LABEL'],ascending=True).reset_index(drop=True)
bob.mongo.save_df(upper_filter_202,'202_UPPER_FILTER',db_app,True)

Dataframe 202 successfully save in database app-wsr-dev in MongoDB. Time: --- 20.78868532180786 secnds ---
Dataframe 202_UPPER_FILTER successfully save in database app-wsr-dev in MongoDB. Time: --- 0.019428491592407227 secnds ---


In [22]:
#-- Economy - Equities
domain203 = save_df(get_trend(db_trend, db_trend_ml, 'EQUITIES'),'203')
# domain203

#Save upper filter
upper_filter = domain203[['META','LABEL']].drop_duplicates()

upper_filter_all = upper_filter.copy()
upper_filter_all['META'] = ' All sectors'

upper_filter_203 = pd.concat([upper_filter,upper_filter_all], axis=0).sort_values(by=['META','LABEL'],ascending=True).reset_index(drop=True)
bob.mongo.save_df(upper_filter_203,'203_UPPER_FILTER',db_app,True)
# upper_filter_203

Dataframe 203 successfully save in database app-wsr-dev in MongoDB. Time: --- 49.10305738449097 secnds ---
Dataframe 203_UPPER_FILTER successfully save in database app-wsr-dev in MongoDB. Time: --- 0.02235126495361328 secnds ---


In [23]:
#-- Economy - Commodities
domain204 = save_df(get_trend(db_trend, db_trend_ml, 'COMMODITIES'),'204')
# domain204

#Save upper filter
upper_filter = domain204[['META','LABEL']].drop_duplicates()

upper_filter_all = upper_filter.copy()
upper_filter_all['META'] = ' All categories'

upper_filter_204 = pd.concat([upper_filter,upper_filter_all], axis=0).sort_values(by=['META','LABEL'],ascending=True).reset_index(drop=True)
bob.mongo.save_df(upper_filter_204,'204_UPPER_FILTER',db_app,True)

Dataframe 204 successfully save in database app-wsr-dev in MongoDB. Time: --- 17.575907707214355 secnds ---
Dataframe 204_UPPER_FILTER successfully save in database app-wsr-dev in MongoDB. Time: --- 0.026545047760009766 secnds ---


In [24]:
done = hc.done(hc_key)
print(done)

End ==> send to https://health.cashstory.com/ping/bfc41c24-d278-4bc8-8d95-767e5da44f59, 2020-04-15
